In [ ]:
train test
test
vis


In [1]:
import torch
import numpy as np

In [6]:
traj=torch.ones(5,12,2)
num_interp=4

In [7]:
    sz = traj.shape
    dense = np.zeros((sz[0], (sz[1] - 1) * (num_interp + 1) + 1, 2)) # num_person* 56(11*5+1),2
    dense[:, :1, :] = traj[:, :1]

In [12]:


    for i in range(num_interp+1):
        ratio = (i + 1) / (num_interp + 1) #1/5 2/5 3/5 4/5 1
        dense[:, i+1::num_interp+1, :] = traj[:, 0:-1] * (1 - ratio) + traj[:, 1:] * ratio

In [22]:

a.shape

(5, 56)

In [21]:
a=np.linalg.norm(dense - dense[0:1,:,:], axis=-1)

In [17]:
mask = a[:, 0] > 0

In [19]:
a[mask].min(axis=0) < 0.2

ValueError: zero-size array to reduction operation minimum which has no identity

In [7]:
a=torch.ones(3,5,2)

In [8]:
b=torch.split(a, 2, dim=1)

In [12]:
kstep_V_pred_ls=torch.ones(20,12,14,2)

In [13]:
kstep_V_pred = np.concatenate([traj for traj in kstep_V_pred_ls], axis=1)

In [16]:
kstep_V_pred.shape

(12, 280, 2)

In [17]:
def seq_to_nodes(seq_):
    max_nodes = seq_.shape[1]  # number of pedestrians in the graph
    seq_ = seq_.squeeze()
    seq_len = seq_.shape[2]
    
    V = np.zeros((seq_len, max_nodes, 2))
    for s in range(seq_len):
        step_ = seq_[:, :, s]
        for h in range(len(step_)): 
            V[s, h, :] = step_[h]
            
    return V.squeeze()

In [20]:
obs_traj=torch.ones(1, 5, 2, 8)

In [21]:
V_x = seq_to_nodes(obs_traj.data.cpu().numpy())

In [28]:
nodes=torch.ones(12, 5, 2)
init_node=torch.ones(8, 5, 2)


In [29]:
np.cumsum(nodes, axis=0).shape

torch.Size([12, 5, 2])

In [26]:
nodes_ = np.cumsum(nodes, axis=0) + init_node

RuntimeError: The size of tensor a (12) must match the size of tensor b (8) at non-singleton dimension 0

In [33]:
predicted_trajs_all=torch.ones(5, 12, 2)
predicted_traj=torch.ones( 12, 2)

In [34]:
def interpolate_traj(traj, num_interp=4):
    '''
    Add linearly interpolated points of a trajectory
    [num_person, 12, 2] 4
    '''
    sz = traj.shape
    dense = np.zeros((sz[0], (sz[1] - 1) * (num_interp + 1) + 1, 2)) # num_person* 56(11*5+1),2
    dense[:, :1, :] = traj[:, :1]

    for i in range(num_interp+1):
        ratio = (i + 1) / (num_interp + 1) #1/5 2/5 3/5 4/5 1
        dense[:, i+1::num_interp+1, :] = traj[:, 0:-1] * (1 - ratio) + traj[:, 1:] * ratio

    return dense

In [35]:
    dense_all = interpolate_traj(predicted_trajs_all, 4)
    dense_ego = interpolate_traj(predicted_traj[None, :], 4)

In [43]:
    ph = predicted_traj.shape[0]
    num_interp = 4
    assert predicted_trajs_all.shape[0] > 1

    dense_all = interpolate_traj(predicted_trajs_all, num_interp) # (5, 56, 2)
    dense_ego = interpolate_traj(predicted_traj[None, :], num_interp) # (1, 56, 2)
    distances = np.linalg.norm(dense_all - dense_ego, axis=-1)  # [num_person, 12 * num_interp]
    mask = distances[:, 0] > 0  # exclude primary agent itself



In [55]:
coll_joint_data_ls=np.array(torch.ones( 5,56))

In [56]:
def stack_dict(data_dict):
    for key, coll_step_data in zip(data_dict.keys(), data_dict.values()):
        data_dict[key] = np.stack(coll_step_data, axis=0)  # [X, 56]
    return data_dict

In [58]:
coll_joint_data_ls

array([[1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1.],
       [1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1

In [59]:
    def coll_data_post_processing(coll_data_bigls):
        coll_raw_ = np.concatenate(coll_data_bigls, axis=0)  # [X, 56]
        coll_step_ = np.mean(coll_raw_, axis=0)  # [56]
        coll_step_ = coll_step_[:-1].reshape(-1, 5).mean(axis=1)  # [11]
        coll_cumulative_ = np.asarray([np.mean(coll_raw_[:, :i * 5 + 6].max(axis=1)) for i in range(11)])  # [11]
        return coll_step_, coll_cumulative_

In [75]:
coll_data_bigls=np.array(torch.ones(3,5,56))

In [74]:
coll_raw_ = np.concatenate(coll_data_bigls, axis=0)

In [76]:
a=coll_data_post_processing(coll_data_bigls)

In [77]:
coll_step_, coll_cumulative_=a

In [80]:
coll_cumulative_

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32)